<a href="https://colab.research.google.com/github/EduMoura321/Non-linear-systems/blob/main/classical_mechanics_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
!pip install --force-reinstall sympy==1.13.3
import sympy as sp
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

sp.init_printing(use_unicode=True)


  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached sympy-1.13.3-py3-none-any.whl (6.2 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Attempting uninstall: mpmath
    Found existing installation: mpmath 1.3.0
    Uninstalling mpmath-1.3.0:
      Successfully uninstalled mpmath-1.3.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3


In [66]:
### Definindo os símbolos pro Sympy
# t: time, g: gravity, M: Cart mass, m: Pendulum mass, l: Pendulum length
t, g, M, m, l = sp.symbols('t g M m l')

# --- Generalized Coordinates (q) ---
# X: cart position, th1: pendulum 1 angle, th2: pendulum 2 angle
X, th1, th2 = sp.symbols('X theta1 theta2')

# --- Generalized Velocities (dq/dt) ---
dX, dth1, dth2 = sp.symbols('dX dtheta1 dtheta2')

# Create lists for easier processing
q = [X, th1, th2]
dq = [dX, dth1, dth2]

In [75]:
### Derivando o Lagrangiano (de acordo com o PDF)

# --- A. Positions of the masses (as per your PDF) ---
P1x = - X - l * sp.sin(th1) # P1x = -X - l*sen(θ1)
P1y = l * sp.cos(th1) # P1y = l*cos(θ1)

P2x = P1x - l * sp.sin(th2) # P2x = -X - l*sen(θ1) - l*sen(θ2)
P2y = P1y + l * sp.cos(th2) # P2y = l*cos(θ1) + l*cos(θ2)

# --- B. Velocities of the masses ---
v1x = P1x.diff(X) * dX + P1x.diff(th1)* dth1
v1y = P1y.diff(th1)* dth1

v2x = P2x.diff(X) * dX + P2x.diff(th1)* dth1 + P2x.diff(th2)* dth2
v2y = P2y.diff(th1)* dth1  + P2y.diff(th2)* dth2
# --- C. Kinetic Energy (T) ---
T_cart = sp.Rational(1, 2) * M * dX**2
T1 = sp.Rational(1, 2) * m * (v1x**2 + v1y**2)
T2 = sp.Rational(1, 2) * m * (v2x**2 + v2y**2)

T = T_cart + T1 + T2
T = sp.simplify(T) # Simplify the (very large) expression - Removed simplify call

# --- D. Potential Energy (V) ---
# V = m*g*y1 + m*g*y2
V = - m * g * P1y - m * g * P2y
V = sp.simplify(V) # This matches your note: 2*m*g*l*cos(th1) + m*g*l*cos(th2) - Removed simplify call

# --- E. Lagrangian (L) ---
L = (T - V)

AttributeError: module 'sympy' has no attribute 'polys'

In [55]:
### Derivando o Hamiltoniano

# --- A. Find Generalized Momenta (p_i) ---
pX = L.diff(dX)
pth1 = L.diff(dth1)
pth2 = L.diff(dth2)

# We also need symbols for the momenta themselves
pX_s, pth1_s, pth2_s = sp.symbols('p_X p_theta1 p_theta2')
p_syms = [pX_s, pth1_s, pth2_s]
q_syms = [X, th1, th2]

# --- B. Solve for Velocities (in terms of momenta) ---
# Create the equations: (expression_for_pX) - pX_symbol = 0
eq_pX = pX - pX_s
eq_pth1 = pth1 - pth1_s
eq_pth2 = pth2 - pth2_s

# Solve the system. SymPy returns a dictionary of solutions.
vel_sols = sp.solve([eq_pX, eq_pth1, eq_pth2], [dX, dth1, dth2])

# --- C. Define the Hamiltonian (H) ---
# H = p*dq - L
H = pX_s * dX + pth1_s * dth1 + pth2_s * dth2 - L

# Substitute the velocity solutions into H
# This replaces (dX, dth1, dth2) with their (p_X, p_th1, p_th2) equivalents
H = H.subs(vel_sols)
H = sp.simplify(H)


AttributeError: module 'sympy' has no attribute 'polys'

In [ ]:
### Obtendo as 6 EDOs (Hamilton)

# --- Find d(q)/dt = ∂H/∂p ---
dq_dt = [sp.simplify(H.diff(p)) for p in p_syms]

# --- Find d(p)/dt = -∂H/∂q ---
dp_dt = [sp.simplify(-H.diff(q)) for q in q_syms]

# This is our final system of 6 first-order ODEs
dZ_dt_sym = dq_dt + dp_dt # [dX, dth1, dth2, dpX, dpth1, dpth2]

In [ ]:
### Simulação Numérica (Lambdify)

# --- A. Create the Numerical Function ---
state_syms = q_syms + p_syms # [X, th1, th2, p_X, p_th1, p_th2]
param_syms = [M, m, l, g]     # [M, m, l, g]

# lambdify creates a fast numerical function from symbolic expressions
dZ_dt_num = sp.lambdify(state_syms + param_syms, dZ_dt_sym, 'numpy')

# --- B. Define the ODE Function for SciPy ---
# solve_ivp expects a function f(t, Z).
def odefunc(t, Z, M_val, m_val, l_val, g_val):
    # Z is a single vector: [X, th1, th2, p_X, p_th1, p_th2]
    # *Z "unpacks" this vector to match the arguments of dZ_dt_num
    return dZ_dt_num(*Z, M_val, m_val, l_val, g_val)

In [ ]:
### Simulação

# --- A. Set Parameters and Initial Conditions ---
params = {
    'M': 10.0,  # kg (cart mass)
    'm': 0.2,  # kg (pendulum mass)
    'l': 0.5,  # m (pendulum length)
    'g': 9.81  # m/s^2
}
param_tuple = (params['M'], params['m'], params['l'], params['g'])

# Initial conditions (Z_0 = [X, th1, th2, pX, pth1, pth2])
# Start with the cart at 0, pendulums slightly off-vertical, all momenta = 0
Z0 = [
    0.01,  # X_0
    0.01,  # theta1_0 (radians)
    0.01, # theta2_0 (radians)
    0.01,  # pX_0
    0.01,  # pth1_0
    0.01   # pth2_0
]

# --- B. Run the Simulation ---
t_span = [0, 100] # Simulate for 10 seconds
t_eval = np.arange(0, 100, 0.001) # Points to get output at

print(f"Resolvendo EDOs pelo t = {t_span[0]}s to {t_span[1]}s...")
sol = solve_ivp(
    odefunc,
    t_span,
    Z0, t_eval=t_eval,
    method='RK45',
    args=param_tuple # Pass parameters to odefunc
)

In [ ]:
### Plots

# --- A. Unpack Results ---
X_t, th1_t, th2_t, pX_t, pth1_t, pth2_t = sol.y


# --- B. Create Plots ---
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Plots Espaço de Fase (Hamiltoniano)', fontsize=16)

# Plot 1: Cart Phase Space
axes[0].plot(X_t, pX_t)
axes[0].set_title('Espaço de Fase: Carrinho')
axes[0].set_ylabel('Momentum $p_X$')
axes[0].set_xlabel('Posição $X$ (Positive is LEFT)') # Note on coord system
axes[0].grid(True)

# Plot 2: Pendulum 1 Phase Space
axes[1].plot(th1_t, pth1_t)
axes[1].set_title('Espaço de Fase: Pendulo 1')
axes[1].set_ylabel('Momentum $p_{\\theta 1}$')
axes[1].set_xlabel('Angulo $\\theta_1$ (rad)')
axes[1].grid(True)

# Plot 3: Pendulum 2 Phase Space
axes[2].plot(th2_t, pth2_t)
axes[2].set_title('Espaço de Fase: Pendulo 2')
axes[2].set_ylabel('Momentum $p_{\\theta 2}$')
axes[2].set_xlabel('Angulo $\\theta_2$ (rad)')
axes[2].grid(True)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Plots Posicao pelo tempo', fontsize=16)
#positions X time:

# Plot 4: Cart Position
axes[0].plot(sol.t,X_t)
axes[0].set_title('Posição Carrinho')
axes[0].set_ylabel('Posição(M)')
axes[0].set_xlabel('Tempo(s)')
axes[0].grid(True)
# Plot 5: Pendulum 1 position
axes[1].plot(sol.t,th1_t)
axes[1].set_title('Pendulo 1')
axes[1].set_ylabel('Angulo $\\theta_2$ (rad)')
axes[1].set_xlabel('Tempo(s)')
axes[1].grid(True)
# Plot 6: Pendulum 2 position
axes[2].plot(sol.t,th2_t)
axes[2].set_title('Pendulo 2')
axes[2].set_ylabel('Angulo $\\theta_1$ (rad)')
axes[2].set_xlabel('Tempo(s)')
axes[2].grid(True)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()